In [ ]:
import gymnasium as gym
import gym_anytrading

from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3 import A2C, PPO

import numpy as np
import pandas as pd
from matplotlib import pylab as plt

# DATA PREPROCESSING

In [ ]:
df = pd.read_csv('./gmedata.csv')
df.head()

In [ ]:
df["Date"] = pd.to_datetime(df["Date"])
df.dtypes

In [ ]:
df.set_index("Date",inplace=True)
df.head()

# SETTING UP ENVIRONMENT AND RANDOM TESTING 

In [ ]:
env = gym.make('stocks-v0', df=df, frame_bound=(5,100), window_size=5)

In [ ]:
env.signal_features

In [ ]:
env.action_space.sample() # Discrete action space 0 and 1 for buy and sell

In [ ]:
state = env.reset()
while True:
    action = env.action_space.sample()
    n_state, reward, terminated, truncated, info = env.step(action)
    if terminated or truncated:
        print(info)
        break

plt.figure(figsize=(15, 6))
plt.cla()
env.render_all()
plt.show()

# TRAINING THE MODEL RIGHT

In [ ]:
env_maker = lambda: gym.make("stocks-v0", df=df, frame_bound=(5, 100), window_size=5)   
env = DummyVecEnv([env_maker])

In [ ]:
model = PPO('MlpLstmPolicy', env, verbose=1)
model.learn(total_timesteps=100000)


In [ ]:
env = gym.make("stocks-v0", df=df, frame_bound=(80, 100), window_size=5)
state = env.reset()

while True:
    state = state[np.newaxis, ...]
    action, _states = model.predict(state)
    state, reward, terminated, truncated, info = env.step(action)
    if terminated or truncated:
        print("Info:", info)
        break

In [ ]:
plt.figure(figsize=(15, 6))
plt.cla()
env.render_all()
plt.show()